In [1]:
%pip install optlang
from optlang import Model, Variable, Constraint, Objective
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


## A Multi-Period Production Model

In [2]:
#creating indexes
T = 4
products = ["bands", "coils"]
availability = [40, 40, 32, 40]
time = range(1, T+1)
#starts at t=0
inv_time = range(0, T+1)

#storing data in dictionaries
inv_0 = {
    "bands": 10, 
    "coils": 0
}
revenue = {
    "bands": [25, 26, 27, 27],
    "coils": [30, 35, 37, 39]
}
market = {
    "bands": [6000, 6000, 4000, 6500],
    "coils": [4000, 2500, 3500, 4200]
}
prod_cost = {
    "bands": 10,
    "coils": 11
}
prod_rate = {
    "bands": 200,
    "coils": 140
}
prod_inv_cost = {
    "bands": 2.5,
    "coils": 3
}

In [3]:
#creating data frames
df_revenue = pd.DataFrame(revenue, index=time)
df_market = pd.DataFrame(market, index=time)
df_prod_cost = pd.DataFrame(prod_cost, index=products)
df_prod_rate = pd.DataFrame(prod_rate, index=products)
df_prod_inv_cost = pd.DataFrame(prod_inv_cost, index=products)

print("revenue data frame:")
df_revenue

revenue data frame:


,bands,coils
1,25,30
2,26,35
3,27,37
4,27,39


In [4]:
print("market data frame:")
df_market

market data frame:


,bands,coils
1,6000,4000
2,6000,2500
3,4000,3500
4,6500,4200


In [5]:
#creating variables in dictionaries
#specific product manufactured in specific time 
prod_made = {}
for i in time:
    prod_made[i] = [Variable(name=f"{product}_made_in_{i}", lb = 0, type='integer') for product in products]

#specific product sold at specific time
prod_sold = {}
for i in time:
    prod_sold[i] = [Variable(name=f"{product}_sold_in_{i}", lb = 0, ub=df_market.loc[i][product],type='integer') for product in products]

#specific product inventoried at specific time
prod_inventory = {}
for i in inv_time:
    prod_inventory[i] = [Variable(name=f"{product}_in_inventory_{i}", lb=0, type='integer') for product in products]

#converting dictionaries to dataframes
df_prod_made = pd.DataFrame(prod_made, index=products)
df_prod_sold = pd.DataFrame(prod_sold, index=products)
df_prod_inventory = pd.DataFrame(prod_inventory, index=products)

In [6]:
#making constraints
constraints = []

#time constraint
constraints.append(list(Constraint(
    expression = sum((1/df_prod_rate.loc[product][product])*df_prod_made.loc[product][i] for product in products), 
    name=f"time_constraint_for_time_{i}",
    ub = availability[i-1]
) for i in time))

#initial inventory constraints
constraints.append(list(Constraint(
    expression=df_prod_inventory.loc[product][0],
    name = f"initial_inventory_constraint_for_product_{product}",
    ub=inv_0[product], lb =inv_0[product]
) for product in products))

#balance constraint
constraints.append(list(Constraint(
    expression=df_prod_made.loc[product][i] + df_prod_inventory.loc[product][i-1] - df_prod_sold.loc[product][i] - df_prod_inventory.loc[product][i], name=f"balance_constraint_for_time_{i}_product_{product}",
     ub = 0, lb = 0
) for i in time for product in products))

In [7]:
#creating objective function
objective = sum(df_revenue.loc[i][product]*df_prod_sold.loc[product][i] - df_prod_cost.loc[product][product]*df_prod_made.loc[product][i] - df_prod_inv_cost[product][product]*df_prod_inventory.loc[product][i] for product in products for i in time)

#creating model, adding obj and constraints and optimizing
model = Model(name="multi_period_production")
model.add(constraints)
model.objective = Objective(expression= objective, direction='max')
status = model.optimize()

#seperating zero and non zero variables from optimal model
zero_vars = list(f"{var.name}: {var.primal}" for var in model.variables if var.primal == 0)
non_zero_vars = {
    "produced": list(f"{var.name}: {var.primal}" for var in model.variables for product in products for i in time if var.primal != 0 if var.name == f"{product}_made_in_{i}"),
    "sold": list(f"{var.name}: {var.primal}" for var in model.variables for product in products for i in time if var.primal != 0 if var.name == f"{product}_sold_in_{i}"),
    "inventory": list(f"{var.name}: {var.primal}" for var in model.variables for product in products for i in inv_time if var.primal != 0 if var.name == f"{product}_in_inventory_{i}")
}

In [8]:
#outputting results
print(f"Model: {model.name}")
print("Status:", status)
print(f"Objective value: ${model.objective.value}\n")
print("non-zero variables:\n")
print("produced:")
print(non_zero_vars["produced"], "\n")
print("sold:")
print(non_zero_vars["sold"], "\n")
print("inventoried: ")
print(non_zero_vars["inventory"], "\n")
print("zero variables:\n")
for var in zero_vars:
    print(var)

Model: multi_period_production
Status: optimal
Objective value: $515033.0

non-zero variables:

produced:
['coils_made_in_1: 1407.0', 'bands_made_in_1: 5990.0', 'coils_made_in_2: 1400.0', 'bands_made_in_2: 6000.0', 'coils_made_in_3: 3500.0', 'bands_made_in_3: 1400.0', 'bands_made_in_4: 2000.0', 'coils_made_in_4: 4200.0'] 

sold:
['bands_sold_in_1: 6000.0', 'coils_sold_in_1: 307.0', 'bands_sold_in_2: 6000.0', 'coils_sold_in_2: 2500.0', 'bands_sold_in_3: 1400.0', 'coils_sold_in_3: 3500.0', 'bands_sold_in_4: 2000.0', 'coils_sold_in_4: 4200.0'] 

inventoried: 
['bands_in_inventory_0: 10.0', 'coils_in_inventory_1: 1100.0'] 

zero variables:

coils_in_inventory_0: 0.0
bands_in_inventory_1: 0.0
bands_in_inventory_2: 0.0
coils_in_inventory_2: 0.0
bands_in_inventory_3: 0.0
coils_in_inventory_3: 0.0
bands_in_inventory_4: 0.0
coils_in_inventory_4: 0.0


The output is identical to the AMPL textbook, hence, we assume the implementation is valid.